# Ensemble Learning

## Initial Imports

In [75]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [77]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder

## Read the CSV and Perform Basic Data Cleaning

In [78]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [ ]:
# Do i need to get dummies for all the cokumns that are not numbers? or do i do a labelencoder for the columns?


In [79]:
# Get dummies for select columns
#df_dummies = pd.get_dummies(df, columns=["home_ownership", "verification_status", "initial_list_status"])
#df_dummies.head()

In [80]:
#label_encoder = LabelEncoder()

In [59]:
# payment plan column
label_encoder.fit(df_dummies["pymnt_plan"])
df_dummies["pymnt_plan"] = label_encoder.transform(df_dummies["pymnt_plan"])

# Encoding hard_ship_flag column
label_encoder.fit(df_dummies["hardship_flag"])
df_dummies["hardship_flag"] = label_encoder.transform(df_dummies["hardship_flag"])

# Encoding debt_settlement_flag column
label_encoder.fit(df_dummies["debt_settlement_flag"])
df_dummies["debt_settlement_flag"] = label_encoder.transform(df_dummies["debt_settlement_flag"])

# Encoding intial_list_status column 
#label_encoder.fit(df_dummies["intial_list_status"])
#df_dummies["intial_list_status"] = label_encoder.transform(df_dummies["intial_list_status"])

# Encoding next_pymnt_d column, this could possibly be dropped??
label_encoder.fit(df_dummies["next_pymnt_d"])
df_dummies["next_pymnt_d"] = label_encoder.transform(df_dummies["next_pymnt_d"])

# Encoding application_type column 
label_encoder.fit(df_dummies["application_type"])
df_dummies["application_type"] = label_encoder.transform(df_dummies["application_type"])



df_dummies.head(20)

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,debt_settlement_flag,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,initial_list_status_f,initial_list_status_w
0,10500.0,0.1719,375.35,66000.0,Mar-2019,low_risk,0,27.24,0.0,0.0,...,0,0,0,0,1,0,1,0,0,1
1,25000.0,0.2000,929.09,105000.0,Mar-2019,low_risk,0,20.23,0.0,0.0,...,0,0,1,0,0,0,0,1,0,1
2,20000.0,0.2000,529.88,56000.0,Mar-2019,low_risk,0,24.26,0.0,0.0,...,0,0,1,0,0,0,0,1,0,1
3,10000.0,0.1640,353.55,92000.0,Mar-2019,low_risk,0,31.44,0.0,1.0,...,0,0,0,0,1,0,0,1,0,1
4,22000.0,0.1474,520.39,52000.0,Mar-2019,low_risk,0,18.76,0.0,1.0,...,0,0,1,0,0,1,0,0,0,1
5,25000.0,0.1797,634.43,90000.0,Mar-2019,low_risk,0,18.49,1.0,1.0,...,0,0,1,0,0,0,1,0,0,1
6,20400.0,0.2000,540.48,51315.0,Mar-2019,low_risk,0,25.63,0.0,2.0,...,0,0,0,0,1,0,1,0,0,1
7,10000.0,0.1557,349.46,66000.0,Mar-2019,low_risk,0,9.00,0.0,0.0,...,0,0,0,1,0,0,1,0,0,1
8,14000.0,0.0881,443.96,45000.0,Mar-2019,low_risk,0,10.24,0.0,1.0,...,0,0,0,0,1,1,0,0,1,0
9,10000.0,0.1474,345.39,63295.0,Mar-2019,low_risk,0,39.34,0.0,0.0,...,0,0,0,0,1,0,0,1,0,1


## Split the Data into Training and Testing

In [81]:
# Create our features
X = df.drop(columns = ["loan_status"])
X = pd.get_dummies(X)

# Create our target
y = df["loan_status"].to_frame()

In [82]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [85]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [87]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

In [88]:
X_test.head(20)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
7034,10000.0,0.1308,337.33,65000.0,33.30,2.0,0.0,7.0,0.0,8474.0,...,1,1,0,1,0,1,1,0,1,1
38804,10275.0,0.1614,250.64,107000.0,17.05,0.0,1.0,17.0,1.0,16893.0,...,0,1,0,1,1,0,1,0,1,1
65533,40000.0,0.0819,1256.97,170000.0,14.33,0.0,0.0,11.0,0.0,27265.0,...,0,1,0,1,0,1,1,0,1,1
37736,9000.0,0.1356,305.68,98000.0,17.17,0.0,1.0,9.0,0.0,12609.0,...,0,1,0,1,1,0,1,0,1,1
6390,9100.0,0.1640,321.73,90000.0,20.92,0.0,1.0,22.0,0.0,13075.0,...,1,1,0,1,0,1,1,0,1,1
62444,8500.0,0.0881,269.55,61000.0,17.23,0.0,0.0,8.0,0.0,8435.0,...,0,1,0,1,0,1,1,0,1,1
60687,15000.0,0.0819,471.37,120000.0,5.09,1.0,1.0,14.0,1.0,899.0,...,0,1,0,1,1,0,1,0,1,1
18124,5000.0,0.1557,174.73,45000.0,22.67,0.0,2.0,9.0,1.0,11691.0,...,0,1,0,1,1,0,1,0,1,1
20534,6500.0,0.1447,223.65,30000.0,23.04,0.0,0.0,8.0,1.0,7637.0,...,0,1,0,1,1,0,1,0,1,1
33256,5500.0,0.0881,174.42,53000.0,18.64,0.0,0.0,10.0,0.0,3448.0,...,0,1,0,1,1,0,1,0,1,1


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [89]:
# Create the StandardScaler instance
from sklearn.preprocessing import  StandardScaler
data_scaler = StandardScaler()

In [90]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
data_scaler.fit(X_train, X_test)

StandardScaler()

In [91]:
# Scale the training and testing data
X_trained_scaled = data_scaler.transform(X_train)
X_test_scaled = data_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [92]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brfc.fit(X_trained_scaled, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [93]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brfc.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7688604483694395

In [94]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   64,    37],
       [ 1641, 15463]], dtype=int64)

In [95]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.63      0.90      0.07      0.76      0.56       101
   low_risk       1.00      0.90      0.63      0.95      0.76      0.59     17104

avg / total       0.99      0.90      0.64      0.94      0.76      0.59     17205



In [97]:
# List the features sorted in descending order by feature importance
importances = brfc.feature_importances_
indices = np.argsort(importances)[::-1]
for i in range(X.shape[1]):
    print(f"{X.columns.values[indices[i]]} : ({importances[indices[i]]})")

total_rec_prncp : (0.06862801182224613)
total_rec_int : (0.06038649619780159)
total_pymnt : (0.05905244774386602)
total_pymnt_inv : (0.05863524144816048)
last_pymnt_amnt : (0.055623377348809976)
int_rate : (0.030111484922129554)
issue_d_Jan-2019 : (0.01777359484825545)
dti : (0.017520298295269406)
installment : (0.017336389337338155)
mths_since_recent_inq : (0.017239635318930423)
out_prncp : (0.016079131330976076)
max_bal_bc : (0.01592943577126096)
out_prncp_inv : (0.015333421064256041)
mo_sin_old_rev_tl_op : (0.015133945030032623)
revol_bal : (0.014958790682464035)
avg_cur_bal : (0.014473955358829452)
total_bal_ex_mort : (0.014369098673870841)
tot_hi_cred_lim : (0.014247071880986605)
issue_d_Mar-2019 : (0.014194503248174256)
annual_inc : (0.014188118477587585)
mths_since_rcnt_il : (0.014131045713741129)
bc_open_to_buy : (0.01394754842726268)
bc_util : (0.013638285505964695)
total_bal_il : (0.013520145899029239)
total_rev_hi_lim : (0.013452985350130738)
total_bc_limit : (0.013426819893

### Easy Ensemble Classifier

In [98]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_trained_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [100]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.931601605553446

In [101]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   93,     8],
       [  985, 16119]], dtype=int64)

In [102]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier had the higher balance score.

2. Which model had the best recall score?

    The Easy Ensemble Classifier had the better recall score.

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier had the better geometric score.

4. What are the top three features?

    The top three features were:
    
    1)total_rec_prncp : (0.06862801182224613)
    
    2)total_rec_int : (0.06038649619780159)
    
    3)total_pymnt : (0.05905244774386602)
        